In [0]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.types import DoubleType, IntegerType

database = "data"  
silver_path = "/Volumes/workspace/data/bda_project/silver"   
gold_base = "/Volumes/workspace/data/bda_project/gold"     


tx = spark.read.format("delta").load(silver_path)


tx = tx.withColumn("amount", F.col("amount").cast(DoubleType()))
tx = tx.withColumn("event_date", F.coalesce(F.to_date("event_date"), F.to_date("event_time")))


kpi_daily = tx.groupBy("event_date").agg(
    F.count("*").cast(IntegerType()).alias("txn_count"),
    F.sum("amount").alias("txn_amount_total"),
    F.avg("amount").alias("txn_amount_avg"),
    F.sum(F.when(F.col("label_fraud") == True, 1).otherwise(0)).alias("fraud_count"),
    F.countDistinct("card_id").alias("unique_cards")
).withColumn(
    "fraud_rate",
    F.when(F.col("txn_count") == 0, F.lit(0.0))
     .otherwise(F.col("fraud_count") / F.col("txn_count"))
)


kpi_daily_path = f"{gold_base}/kpi_daily"
kpi_daily.write.format("delta").mode("overwrite").option("overwriteSchema", "true").partitionBy("event_date").save(kpi_daily_path)
kpi_daily.coalesce(1).write.mode("overwrite").option("header", True).csv(f"{gold_base}/csv/kpi_daily.csv")


w7 = Window.orderBy(F.col("event_date").cast("timestamp")).rowsBetween(-6, 0)
kpi_daily_ordered = kpi_daily.select("event_date", "txn_count").orderBy("event_date")
kpi_7day = kpi_daily_ordered.withColumn("txn_count_7d_sum", F.sum("txn_count").over(w7)) \
                            .withColumn("txn_count_7d_avg", F.avg("txn_count").over(w7))

kpi_7day_path = f"{gold_base}/kpi_7day_txn"
kpi_7day.write.format("delta").mode("overwrite").option("overwriteSchema", "true").partitionBy("event_date").save(kpi_7day_path)
kpi_7day.coalesce(1).write.mode("overwrite").option("header", True).csv(f"{gold_base}/csv/kpi_7day_txn.csv")

# -----------------------------------
# KPI 3 — Merchant category performance
# -----------------------------------
kpi_merchant = tx.groupBy("merchant_cat").agg(
    F.count("*").alias("txn_count"),
    F.avg("amount").alias("avg_amount"),
    F.sum(F.when(F.col("label_fraud"), 1).otherwise(0)).alias("fraud_count"),
    F.sum(F.when(F.col("is_contactless"), 1).otherwise(0)).alias("contactless_count"),
    F.sum(F.when(F.col("is_chip"), 1).otherwise(0)).alias("chip_count")
).withColumn(
    "fraud_rate",
    F.when(F.col("txn_count") == 0, F.lit(0.0))
     .otherwise(F.col("fraud_count") / F.col("txn_count"))
).withColumn(
    "pct_contactless",
    F.when(F.col("txn_count") == 0, F.lit(0.0))
     .otherwise(F.col("contactless_count") / F.col("txn_count"))
).withColumn(
    "pct_chip",
    F.when(F.col("txn_count") == 0, F.lit(0.0))
     .otherwise(F.col("chip_count") / F.col("txn_count"))
)

kpi_merchant_path = f"{gold_base}/kpi_merchant_cat"
kpi_merchant.write.format("delta").mode("overwrite").option("overwriteSchema", "true").save(kpi_merchant_path)
kpi_merchant.coalesce(1).write.mode("overwrite").option("header", True).csv(f"{gold_base}/csv/kpi_merchant_cat.csv")

# -----------------------------------
# KPI 4 — City-level international exposure
# -----------------------------------
kpi_city = tx.groupBy("city").agg(
    F.count("*").alias("txn_count"),
    F.sum(F.when(F.col("is_international"), 1).otherwise(0)).alias("international_count"),
    F.avg("amount").alias("avg_amount"),
    F.sum(F.when(F.col("label_fraud"), 1).otherwise(0)).alias("fraud_count")
).withColumn(
    "pct_international",
    F.when(F.col("txn_count") == 0, F.lit(0.0))
     .otherwise(F.col("international_count") / F.col("txn_count"))
).withColumn(
    "fraud_rate",
    F.when(F.col("txn_count") == 0, F.lit(0.0))
     .otherwise(F.col("fraud_count") / F.col("txn_count"))
)

kpi_city_path = f"{gold_base}/kpi_city_international"
kpi_city.write.format("delta").mode("overwrite").option("overwriteSchema", "true").save(kpi_city_path)
kpi_city.coalesce(1).write.mode("overwrite").option("header", True).csv(f"{gold_base}/csv/kpi_city_international.csv")

# -----------------------------------
# KPI 5 — Top 10 cards by total transaction amount
# -----------------------------------
kpi_card = tx.groupBy("card_id").agg(
    F.count("*").alias("txn_count"),
    F.sum("amount").alias("txn_amount_total"),
    F.sum(F.when(F.col("label_fraud"), 1).otherwise(0)).alias("fraud_count")
).withColumn(
    "fraud_rate",
    F.when(F.col("txn_count") == 0, F.lit(0.0))
     .otherwise(F.col("fraud_count") / F.col("txn_count"))
)

kpi_card_top10 = kpi_card.orderBy(F.col("txn_amount_total").desc()).limit(10)
kpi_card_path = f"{gold_base}/kpi_top_cards"
kpi_card_top10.write.format("delta").mode("overwrite").option("overwriteSchema", "true").save(kpi_card_path)
kpi_card_top10.coalesce(1).write.mode("overwrite").option("header", True).csv(f"{gold_base}/csv/kpi_top_cards.csv")

# -----------------------------------
# ✅ Register in Unity Catalog (optional but recommended)
# -----------------------------------
# spark.sql(f"CREATE DATABASE IF NOT EXISTS {database}")
# spark.sql(f"CREATE TABLE IF NOT EXISTS {database}.kpi_daily USING DELTA LOCATION '{kpi_daily_path}'")
# spark.sql(f"CREATE TABLE IF NOT EXISTS {database}.kpi_7day_txn USING DELTA LOCATION '{kpi_7day_path}'")
# spark.sql(f"CREATE TABLE IF NOT EXISTS {database}.kpi_merchant_cat USING DELTA LOCATION '{kpi_merchant_path}'")
# spark.sql(f"CREATE TABLE IF NOT EXISTS {database}.kpi_city_international USING DELTA LOCATION '{kpi_city_path}'")
# spark.sql(f"CREATE TABLE IF NOT EXISTS {database}.kpi_top_cards USING DELTA LOCATION '{kpi_card_path}'")

# 🟢 Done
print("✅ All 5 KPIs computed and saved to Gold + CSV:")
print(f" - {kpi_daily_path}")
print(f" - {kpi_7day_path}")
print(f" - {kpi_merchant_path}")
print(f" - {kpi_city_path}")
print(f" - {kpi_card_path}")


/databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/expressions.py:1117: UserWarning: WARN WindowExpression: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
  warnings.warn(


✅ All 5 KPIs computed and saved to Gold + CSV:
 - /Volumes/workspace/data/bda_project/gold/kpi_daily
 - /Volumes/workspace/data/bda_project/gold/kpi_7day_txn
 - /Volumes/workspace/data/bda_project/gold/kpi_merchant_cat
 - /Volumes/workspace/data/bda_project/gold/kpi_city_international
 - /Volumes/workspace/data/bda_project/gold/kpi_top_cards


In [0]:
kpi_daily_df = spark.read.csv(
    "/Volumes/workspace/data/bda_project/gold/csv/kpi_daily.csv",  # path to the folder
    header=True,                          # use the first row as column names
    inferSchema=True                       # automatically detect data types
)
display(kpi_daily_df)


event_date,txn_count,txn_amount_total,txn_amount_avg,fraud_count,unique_cards,fraud_rate
2025-01-30,9206,15265.38766162729,1.6581998328945569,89,9206,0.009667608081685857
2025-01-08,8988,14957.368019464811,1.6641486448002683,86,8986,0.009568313306631064
2025-02-03,9031,14801.99406082301,1.6390204917310387,89,9031,0.009854944081497065
2025-02-22,7609,12543.962981502653,1.6485691919440995,61,7609,0.008016822184255487
2025-01-04,7446,12315.809037871815,1.6540167926231286,70,7445,0.009401020682245501
2025-02-12,9123,15317.508128858759,1.6789990276070108,74,9123,0.008111366874931492
2025-02-28,9089,14860.859125251229,1.635037861728598,88,9089,0.009682033226977665
2025-01-05,6929,11418.010286172654,1.647858318108335,63,6929,0.00909222109972579
2025-02-19,9010,14854.893203007621,1.6487117872372499,79,9010,0.00876803551609323
2025-01-10,9185,15091.076352499036,1.6430132120303795,81,9185,0.008818726183995646


Databricks visualization. Run in Databricks to view.

In [0]:
kpi_7day_df = spark.read.csv(
    "/Volumes/workspace/data/bda_project/gold/csv/kpi_7day_txn.csv/",  # path to the folder
    header=True,                          # use the first row as column names
    inferSchema=True                       # automatically detect data types
)
display(kpi_7day_df)


event_date,txn_count,txn_count_7d_sum,txn_count_7d_avg
2025-01-01,6757,6757,6757.0
2025-01-02,9052,15809,7904.5
2025-01-03,9072,24881,8293.666666666666
2025-01-04,7446,32327,8081.75
2025-01-05,6929,39256,7851.2
2025-01-06,9218,48474,8079.0
2025-01-07,9008,57482,8211.714285714286
2025-01-08,8988,59713,8530.42857142857
2025-01-09,8940,59601,8514.42857142857
2025-01-10,9185,59714,8530.57142857143


Databricks visualization. Run in Databricks to view.

In [0]:
city_international_df = spark.read.csv(
    "/Volumes/workspace/data/bda_project/gold/csv/kpi_city_international.csv",  # path to the folder
    header=True,                          # use the first row as column names
    inferSchema=True                       # automatically detect data types
)
display(city_international_df)

city,txn_count,international_count,avg_amount,fraud_count,pct_international,fraud_rate
Chennai,99831,7896,1.6510677819867825,974,0.07909366829942603,0.009756488465506707
Delhi,99385,7897,1.648446010051123,1007,0.0794586708255773,0.010132313729436031
Mumbai,100486,8038,1.6594508996214294,935,0.0799912425611528,0.009304778775152758
Bengaluru,150562,12125,1.6565062739157521,1459,0.08053160824112326,0.009690360117426708
Hyderabad,49736,3916,1.6644138287395835,459,0.07873572462602542,0.009228727682161814


Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

In [0]:
kpi_merchant_df = spark.read.csv(
    "/Volumes/workspace/data/bda_project/gold/csv/kpi_merchant_cat.csv/",  # path to the folder
    header=True,                          # use the first row as column names
    inferSchema=True                       # automatically detect data types
)
display(kpi_merchant_df)


merchant_cat,txn_count,avg_amount,fraud_count,contactless_count,chip_count,fraud_rate,pct_contactless,pct_chip
Travel,83465,1.657953668759926,799,33487,58283,0.009572874857724794,0.40121008806086383,0.6982926975378901
BillPay,83358,1.6526115913530375,820,33504,58235,0.00983708822188632,0.4019290290074138,0.698613210489695
Dining,83329,1.6601951601961726,816,33406,58214,0.009792509210478945,0.40089284642801426,0.6986043274250261
Fuel,83626,1.6613264723079475,837,33253,58718,0.010008848922583885,0.39763949011073113,0.7021500490278143
Grocery,83251,1.648265605147236,784,33230,58211,0.009417304296645085,0.399154364512138,0.6992228321581723
Electronics,82971,1.6507764325478806,778,33305,57878,0.009376770196815754,0.4014053102891372,0.697569030143062


Databricks visualization. Run in Databricks to view.

In [0]:
kpi_top_cards_df = spark.read.csv(
    "/Volumes/workspace/data/bda_project/gold/csv/kpi_top_cards.csv/",  # path to the folder
    header=True,                          # use the first row as column names
    inferSchema=True                       # automatically detect data types
)
display(kpi_top_cards_df)


card_id,txn_count,txn_amount_total,fraud_count,fraud_rate
CARD-21678882,1,109.46250593225449,0,0.0
CARD-92084648,1,108.42130078536441,0,0.0
CARD-23536792,1,94.14974525801294,0,0.0
CARD-48603990,1,76.9911062676673,0,0.0
CARD-71849390,1,74.46163496354002,0,0.0
CARD-47850931,1,74.38397963361484,0,0.0
CARD-89572485,1,68.82630097709661,0,0.0
CARD-62711192,1,61.83721906074922,0,0.0
CARD-30116550,1,61.76231492809488,0,0.0
CARD-29801287,1,61.560435277250626,0,0.0


Databricks visualization. Run in Databricks to view.